# Sparl ML 을 이용한 선형 회귀

캘리포니아 주택 데이터셋은 미국 1990년 인구조사를 기반으로 하며, 각 컬럼은 주택, 인구, 지역 등과 관련된 다양한 특성을 나타냅니다. 아래는 데이터셋의 각 컬럼에 대한 상세 설명입니다:

    `MedInc` : 블록 그룹 내 중위 소득 (Median income in block group).  
    `HouseAge` :  블록 그룹 내 중위 주택 연령 (Median house age in block group).  
    `AveRooms`: 가구당 평균 방 개수 (Average number of rooms per household).  
    `AveBedrms`: 가구당 평균 침실 개수 (Average number of bedrooms per household).  
    `Population`: 블록 그룹 내 인구 수 (Population in block group).  
    `AveOccup`: 가구당 평균 구성원 수 (Average number of household members).  
    `Latitude`: 블록 그룹의 위도 (Block group latitude).  
    `Longitude` : 블록 그룹의 경도 (Block group longitude).  
    `MedHouseVal`: 블록 그룹의 중위 주택 가치 (Median house value for block group).  

### 데이터셋 참고사항
    블록 그룹: 인구조사에서 사용되는 최소 지리적 단위로, 600~3,000명의 인구를 포함.
    가구: 한 주택 내에서 거주하는 사람들의 그룹.

이 데이터는 캘리포니아 주택 가격을 예측하는 회귀 문제에 자주 사용되며, 특성 간의 상관관계와 데이터를 기반으로 다양한 패턴을 분석할 수 있습니다.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("LinearRegrssion") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

In [2]:
from sklearn.datasets import fetch_california_housing
import pandas as pd
import numpy as np
# PySpark 선형 회귀 모델
from pyspark.ml.regression import LinearRegression
# 문자열 데이터를 인덱스 번호로 변환
from pyspark.ml.feature import StringIndexer
# 피처를 벡터 형식으로 변환
from pyspark.ml.feature import VectorAssembler
import warnings
warnings.filterwarnings('ignore')

# 캘리포니아 주택 데이터셋 로드
housing = fetch_california_housing()
housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [3]:
# 캘리포니아 주택 데이터를 Pandas 데이터프레임으로 변환
df = pd.DataFrame(
    data=housing.data,           # 입력 데이터 (특성값)
    columns=housing.feature_names       # 컬럼 이름 (특성 이름)
)

df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [4]:
# 타겟 변수(주택 가격)를 데이터프레임에 새로운 열로 추가
df['PRICE'] = housing.target

df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,PRICE
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [5]:
# Pandas DataFrame을 Spark DataFrame으로 변환
spark_df = spark.createDataFrame(df)
spark_df.limit(5)

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,PRICE
8.3252,41.0,6.984126984126984,1.0238095238095237,322.0,2.5555555555555554,37.88,-122.23,4.526
8.3014,21.0,6.238137082601054,0.9718804920913884,2401.0,2.109841827768014,37.86,-122.22,3.585
7.2574,52.0,8.288135593220339,1.073446327683616,496.0,2.8022598870056497,37.85,-122.24,3.521
5.6431,52.0,5.8173515981735155,1.0730593607305936,558.0,2.547945205479452,37.85,-122.25,3.413
3.8462,52.0,6.281853281853282,1.0810810810810811,565.0,2.1814671814671813,37.85,-122.25,3.422


In [6]:
print(spark_df.columns[:-1])

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


### PySpark 데이터프레임에서 머신러닝 모델에 사용할 피처를 벡터화
- VectorAssembler: 다수의 열을 단일 벡터 형식으로 결합하여 머신러닝 모델에 입력으로 사용  

    - transform() - VectorAssembler를 사용해 지정한 컬럼들을 변환합니다. 기존 데이터프레임에 "features"라는 새 컬럼이 추가됩니다.


In [7]:
# Spark ML의 VectorAssembler를 사용하여 피처 벡터 생성
# - inputCols: 입력으로 사용할 컬럼들 지정 (PRICE를 제외한 모든 컬럼)
# - outputCol: 벡터화된 피처가 저장될 새로운 컬럼 이름 ("features")
featureAssembler = VectorAssembler(
    inputCols=spark_df.columns[:-1],
    outputCol="features"
)

# 데이터프레임에 변환 적용하여 "features" 열 추가
transformed_df = featureAssembler.transform(spark_df)

# 결과 데이터프레임에서 상위 5개 행 확인
transformed_df.limit(5)

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,PRICE,features
8.3252,41.0,6.984126984126984,1.0238095238095237,322.0,2.5555555555555554,37.88,-122.23,4.526,"[8.3252,41.0,6.98..."
8.3014,21.0,6.238137082601054,0.9718804920913884,2401.0,2.109841827768014,37.86,-122.22,3.585,"[8.3014,21.0,6.23..."
7.2574,52.0,8.288135593220339,1.073446327683616,496.0,2.8022598870056497,37.85,-122.24,3.521,"[7.2574,52.0,8.28..."
5.6431,52.0,5.8173515981735155,1.0730593607305936,558.0,2.547945205479452,37.85,-122.25,3.413,"[5.6431,52.0,5.81..."
3.8462,52.0,6.281853281853282,1.0810810810810811,565.0,2.1814671814671813,37.85,-122.25,3.422,"[3.8462,52.0,6.28..."


In [8]:
transformed_df.select('features').first()

Row(features=DenseVector([8.3252, 41.0, 6.9841, 1.0238, 322.0, 2.5556, 37.88, -122.23]))

In [9]:
# PySpark ML의 StandardScaler를 사용하여 피처 스케일링 (표준화)
from pyspark.ml.feature import StandardScaler

# StandardScaler 객체 생성
# - inputCol: 표준화를 적용할 입력 벡터 컬럼("features")
# - outputCol: 표준화된 데이터를 저장할 출력 벡터 컬럼("standardized")
scaler = StandardScaler(
    inputCol="features",
    outputCol="standardized"
)

# StandardScaler 모델 학습 및 데이터 변환
data_scaled = scaler.fit(transformed_df).transform(transformed_df)

data_scaled.limit(5)

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,PRICE,features,standardized
8.3252,41.0,6.984126984126984,1.0238095238095237,322.0,2.5555555555555554,37.88,-122.23,4.526,"[8.3252,41.0,6.98...",[4.38209539419521...
8.3014,21.0,6.238137082601054,0.9718804920913884,2401.0,2.109841827768014,37.86,-122.22,3.585,"[8.3014,21.0,6.23...",[4.36956790291791...
7.2574,52.0,8.288135593220339,1.073446327683616,496.0,2.8022598870056497,37.85,-122.24,3.521,"[7.2574,52.0,8.28...",[3.82004265529144...
5.6431,52.0,5.8173515981735155,1.0730593607305936,558.0,2.547945205479452,37.85,-122.25,3.413,"[5.6431,52.0,5.81...",[2.97033134567133...
3.8462,52.0,6.281853281853282,1.0810810810810811,565.0,2.1814671814671813,37.85,-122.25,3.422,"[3.8462,52.0,6.28...",[2.02450575423457...


In [10]:
print(data_scaled.select('features').first())
print(data_scaled.select('standardized').first())

Row(features=DenseVector([8.3252, 41.0, 6.9841, 1.0238, 322.0, 2.5556, 37.88, -122.23]))
Row(standardized=DenseVector([4.3821, 3.2577, 2.8228, 2.1603, 0.2843, 0.2461, 17.7345, -61.0073]))


In [11]:
# 최종 데이터 준비
# - 표준화된 피처("standardized")와 타겟 변수("PRICE")만 선택하여 최종 데이터 생성
ml_data = data_scaled.select("standardized", "PRICE")

# 최종 데이터프레임의 상위 5개 행 출력 (값을 잘리지 않도록 설정: truncate=False)
ml_data.show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|standardized                                                                                                                                             |PRICE|
+---------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|[4.382095394195218,3.257702301608306,2.8228125480951616,2.1603419907541483,0.28433622088661975,0.2460565530953312,17.734477624640157,-61.007269596069534]|4.526|
|[4.36956790291791,1.6685792276530347,2.5213017566153453,2.0507664641049517,2.1201592122632733,0.20314189867185037,17.72511412008649,-61.00227840981444]  |3.585|
|[3.8200426552914495,4.131719992283705,3.3498607923408183,2.265080684038642,0.4379837439744205,0.26980998600283923,17.72043236780966,-61.01226078232463]  |3.521|
|[2.9703313456713394,4.13171

In [12]:
# 데이터를 학습용(train)과 테스트용(test)으로 분할
# randomSplit 함수는 지정된 비율에 따라 데이터를 랜덤하게 분할
# 75%를 학습 데이터로, 25%를 테스트 데이터로 분할
train_data, test_data = ml_data.randomSplit([0.75, 0.25])

train_data.count(), test_data.count()

(15520, 5120)

In [13]:
# pyspark.ml 라이브러리를 사용하여 선형 회귀 모델을 생성
#  "standardized" 컬럼을 특징으로, "PRICE" 컬럼을 라벨로 사용
# fit 메서드를 호출하여 학습 데이터를 기반으로 선형 회귀 모델을 학습
regressor = LinearRegression(featuresCol="standardized", labelCol='PRICE')
model = regressor.fit(train_data)

In [14]:
# 모델의 계수와 절편 출력
print(f"Coefficients: {model.coefficients}")
print(f"Intercept: {model.intercept}")

# 모델 요약 정보 확인
summary = model.summary
print(f"R-squared: {summary.r2}")
print(f"Mean Squared Error: {summary.meanSquaredError}")
print(f"Root Mean Squared Error: {summary.rootMeanSquaredError}")

Coefficients: [0.8247763075101192,0.12196681508179726,-0.24124745498301975,0.27862885338042465,-0.005270282764335624,-0.04754754684103922,-0.9030766680949593,-0.8740091078488724]
Intercept: -37.075780320972264
R-squared: 0.6086164175461264
Mean Squared Error: 0.5266628502828017
Root Mean Squared Error: 0.7257154058463977


In [15]:
# 학습된 선형 회귀 모델을 사용하여 테스트 데이터에 대한 예측 수행
# transform 메서드는 테스트 데이터셋(test_data)을 입력으로 받아 각 샘플의 예측값을 계산
pred = model.transform(test_data)
pred.limit(5)

standardized,PRICE,prediction
[0.26312995334144...,0.735,1.650431204986397
[0.26312995334144...,5.00001,0.8143525708178956
[0.26312995334144...,5.00001,0.9562436997920045
[0.28213173632929...,0.875,0.4824574808765263
[0.35661240925950...,3.5,1.4355919037893656


In [16]:
## spark DataFrame을 numpy array로 변경
y_test = np.array(test_data.select('PRICE').collect())
y_pred = np.array(pred.select('prediction').collect())

In [17]:
from sklearn.metrics import r2_score
# R2 계산
print("결정계수: {:.2f}".format(r2_score(y_test, y_pred)))

결정계수: 0.60


## 2. Linear Regression - Tip 금액 예측

In [18]:
import seaborn as sns

# tips 데이터셋은 식사 팁과 관련된 정보를 포함한 예제 데이터셋
# load_dataset 메서드를 사용하여 "tips"라는 이름의 내장 데이터셋 로드
tips = sns.load_dataset("tips")
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [19]:
## spark dataframe으로 변환
df = spark.createDataFrame(tips)
df.limit(5)

total_bill,tip,sex,smoker,day,time,size
16.99,1.01,Female,No,Sun,Dinner,2
10.34,1.66,Male,No,Sun,Dinner,3
21.01,3.5,Male,No,Sun,Dinner,3
23.68,3.31,Male,No,Sun,Dinner,2
24.59,3.61,Female,No,Sun,Dinner,4


In [20]:
# 'time' 컬럼을 기준으로 그룹화한 후, 각 그룹의 개수를 계산하고 출력
# 어떤 시간대의 데이터가 더 많은지 알 수 있음.
df.groupBy('time').count().show()

# 'smoker' 컬럼을 기준으로 그룹화한 후, 각 그룹의 개수를 계산하고 출력
# 흡연자와 비흡연자의 비율을 확인
df.groupBy('smoker').count().show()

#  'day' 컬럼을 기준으로 그룹화한 후, 각 그룹의 개수를 계산하고 출력
# 특정 요일에 데이터가 더 집중되어 있는지 확인
df.groupBy('day').count().show()

+------+-----+
|  time|count|
+------+-----+
| Lunch|   68|
|Dinner|  176|
+------+-----+

+------+-----+
|smoker|count|
+------+-----+
|    No|  151|
|   Yes|   93|
+------+-----+

+----+-----+
| day|count|
+----+-----+
|Thur|   62|
| Sun|   76|
| Sat|   87|
| Fri|   19|
+----+-----+



### Categorical Feature를 One-Hot 벡터로 변환

One-Hot Encoding은 **범주형 데이터(Categorical Data)**를 숫자 데이터로 변환하는 기법. 각 고유한 범주(category)를 **이진 벡터(binary vector)**로 표현.

예) `['red', 'green', 'blue']` 라는 범주형 데이터
```
red → [1, 0, 0]
green → [0, 1, 0]
blue → [0, 0, 1]
```

- StringIndexer는 문자열 데이터를 숫자형 데이터로 변환: 여러 개의 문자열형 컬럼을 한 번에 숫자형(index)로 변환
- OneHotEncoder는 숫자형 데이터를 벡터 형태로 변환하여 머신러닝 모델에서 사용할 수 있도록 합니다.

In [21]:
# inputCols: 변환할 문자열 컬럼들의 리스트 (여기서는 'sex', 'smoker', 'day', 'time' 컬럼).
# outputCols: 변환된 값을 저장할 새 컬럼들의 리스트 (여기서는 'sex_', 'smoker_', 'day_', 'time_' 컬럼).
# fit 메서드를 호출하여 데이터프레임(df)에 기반한 변환 모델을 학습
indexer = StringIndexer(inputCols=['sex', 'smoker', 'day', 'time'], outputCols=['sex_', 'smoker_', 'day_', 'time_']).fit(df)

# 학습된 StringIndexer 모델을 사용하여 원본 데이터프레임(df)에 변환을 적용
# 변환된 데이터는 df_r 변수에 저장되며, 새로 추가된 컬럼들이 포함
df_r = indexer.transform(df)

# 변환된 데이터프레임(df_r)의 상위 100개 행을 출력하여 변환 결과 확인
df_r.limit(5)

total_bill,tip,sex,smoker,day,time,size,sex_,smoker_,day_,time_
16.99,1.01,Female,No,Sun,Dinner,2,1.0,0.0,1.0,0.0
10.34,1.66,Male,No,Sun,Dinner,3,0.0,0.0,1.0,0.0
21.01,3.5,Male,No,Sun,Dinner,3,0.0,0.0,1.0,0.0
23.68,3.31,Male,No,Sun,Dinner,2,0.0,0.0,1.0,0.0
24.59,3.61,Female,No,Sun,Dinner,4,1.0,0.0,1.0,0.0


In [22]:
df_r.select(['sex_']).distinct().show()
df_r.select(['smoker_']).distinct().show()
df_r.select(['day_']).distinct().show()
df_r.select(['time_']).distinct().show()

+----+
|sex_|
+----+
| 0.0|
| 1.0|
+----+

+-------+
|smoker_|
+-------+
|    0.0|
|    1.0|
+-------+

+----+
|day_|
+----+
| 0.0|
| 1.0|
| 3.0|
| 2.0|
+----+

+-----+
|time_|
+-----+
|  0.0|
|  1.0|
+-----+



In [23]:
from pyspark.ml.feature import OneHotEncoder

# 문자열 데이터를 숫자형으로 변환한 컬럼(sex_, smoker_, day_, time_)을
# 원-핫 인코딩(One-Hot Encoding)으로 변환
# inputCols: 원-핫 인코딩을 적용할 입력 컬럼들의 리스트
# outputCols: 변환된 원-핫 인코딩 결과를 저장할 컬럼들의 리스트
# fit 메서드를 호출하여 데이터프레임(df_r)에 기반한 변환 모델을 학습
ohe_encoder = OneHotEncoder(
    inputCols=['sex_', 'smoker_', 'day_', 'time_'],
    outputCols=['sex_ohe', 'smoker_ohe', 'day_ohe', 'time_ohe']
).fit(df_r)

# 학습된 OneHotEncoder 모델을 사용하여 원-핫 인코딩을 데이터프레임(df_r)에 적용합니다.
# 변환된 데이터는 df_ohe 변수에 저장되며, 새로 추가된 원-핫 인코딩 컬럼들이 포함됩니다.
df_ohe = ohe_encoder.transform(df_r)

df_ohe.limit(10)

total_bill,tip,sex,smoker,day,time,size,sex_,smoker_,day_,time_,sex_ohe,smoker_ohe,day_ohe,time_ohe
16.99,1.01,Female,No,Sun,Dinner,2,1.0,0.0,1.0,0.0,"(1,[],[])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])"
10.34,1.66,Male,No,Sun,Dinner,3,0.0,0.0,1.0,0.0,"(1,[0],[1.0])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])"
21.01,3.5,Male,No,Sun,Dinner,3,0.0,0.0,1.0,0.0,"(1,[0],[1.0])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])"
23.68,3.31,Male,No,Sun,Dinner,2,0.0,0.0,1.0,0.0,"(1,[0],[1.0])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])"
24.59,3.61,Female,No,Sun,Dinner,4,1.0,0.0,1.0,0.0,"(1,[],[])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])"
25.29,4.71,Male,No,Sun,Dinner,4,0.0,0.0,1.0,0.0,"(1,[0],[1.0])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])"
8.77,2.0,Male,No,Sun,Dinner,2,0.0,0.0,1.0,0.0,"(1,[0],[1.0])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])"
26.88,3.12,Male,No,Sun,Dinner,4,0.0,0.0,1.0,0.0,"(1,[0],[1.0])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])"
15.04,1.96,Male,No,Sun,Dinner,2,0.0,0.0,1.0,0.0,"(1,[0],[1.0])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])"
14.78,3.23,Male,No,Sun,Dinner,2,0.0,0.0,1.0,0.0,"(1,[0],[1.0])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])"


In [24]:
# 여러 개의 입력 컬럼을 하나의 벡터로 결합하여 머신러닝 모델에 사용할 수 있도록 변환하는 코드입니다.
# VectorAssembler는 여러 개의 숫자형 컬럼을 단일 feature 벡터로 병합합니다.
# inputCols: 벡터로 결합할 입력 컬럼들의 리스트.
# outputCol: 결합된 벡터를 저장할 출력 컬럼의 이름.
feature_Assembler = VectorAssembler(
    inputCols=['total_bill', 'size', 'sex_ohe', 'smoker_ohe', 'day_ohe', 'time_ohe'],
    outputCol='features'
)

# 학습된 VectorAssembler를 사용하여 데이터프레임(df_ohe)에 변환을 적용
# 입력 컬럼의 데이터를 결합한 결과는 새로운 컬럼 'features'에 저장됩니다.
transformed_ohe = feature_Assembler.transform(df_ohe)

# 변환된 데이터프레임(output)의 상위 행을 출력하여 결과를 확인
transformed_ohe.limit(5)

total_bill,tip,sex,smoker,day,time,size,sex_,smoker_,day_,time_,sex_ohe,smoker_ohe,day_ohe,time_ohe,features
16.99,1.01,Female,No,Sun,Dinner,2,1.0,0.0,1.0,0.0,"(1,[],[])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])","[16.99,2.0,0.0,1...."
10.34,1.66,Male,No,Sun,Dinner,3,0.0,0.0,1.0,0.0,"(1,[0],[1.0])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])","[10.34,3.0,1.0,1...."
21.01,3.5,Male,No,Sun,Dinner,3,0.0,0.0,1.0,0.0,"(1,[0],[1.0])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])","[21.01,3.0,1.0,1...."
23.68,3.31,Male,No,Sun,Dinner,2,0.0,0.0,1.0,0.0,"(1,[0],[1.0])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])","[23.68,2.0,1.0,1...."
24.59,3.61,Female,No,Sun,Dinner,4,1.0,0.0,1.0,0.0,"(1,[],[])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])","[24.59,4.0,0.0,1...."


In [25]:
# 피처 스케일링(feature scaling)을 수행하여 데이터를 표준화(Standardization)
scaler = StandardScaler(inputCol="features", outputCol="standardized")

# 학습 데이터(output)를 기반으로 스케일링 모델을 학습(fit)합니다.
data_scaled = scaler.fit(transformed_ohe).transform(transformed_ohe)

data_scaled.limit(5)

total_bill,tip,sex,smoker,day,time,size,sex_,smoker_,day_,time_,sex_ohe,smoker_ohe,day_ohe,time_ohe,features,standardized
16.99,1.01,Female,No,Sun,Dinner,2,1.0,0.0,1.0,0.0,"(1,[],[])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])","[16.99,2.0,0.0,1....",[1.90847155648990...
10.34,1.66,Male,No,Sun,Dinner,3,0.0,0.0,1.0,0.0,"(1,[0],[1.0])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])","[10.34,3.0,1.0,1....",[1.16148298376136...
21.01,3.5,Male,No,Sun,Dinner,3,0.0,0.0,1.0,0.0,"(1,[0],[1.0])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])","[21.01,3.0,1.0,1....",[2.36003457338745...
23.68,3.31,Male,No,Sun,Dinner,2,0.0,0.0,1.0,0.0,"(1,[0],[1.0])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])","[23.68,2.0,1.0,1....",[2.65995329356568...
24.59,3.61,Female,No,Sun,Dinner,4,1.0,0.0,1.0,0.0,"(1,[],[])","(1,[0],[1.0])","(3,[1],[1.0])","(1,[0],[1.0])","[24.59,4.0,0.0,1....",[2.76217278246537...


In [26]:
print(data_scaled.select('features').first())
print(data_scaled.select('standardized').first())

Row(features=DenseVector([16.99, 2.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0]))
Row(standardized=DenseVector([1.9085, 2.1028, 0.0, 2.0548, 0.0, 2.1549, 0.0, 2.2258]))


In [27]:
# "standardized" 컬럼(표준화된 특징 벡터)과 "tip" 컬럼(타겟 변수)을 선택
data = data_scaled.select("standardized", "tip")

data.show(5, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------+----+
|standardized                                                                                                             |tip |
+-------------------------------------------------------------------------------------------------------------------------+----+
|[1.9084715564899044,2.1028287357950464,0.0,2.0547931158599058,0.0,2.1549470973741336,0.0,2.225805748475487]              |1.01|
|[1.1614829837613663,3.1542431036925693,2.083476457489454,2.0547931158599058,0.0,2.1549470973741336,0.0,2.225805748475487]|1.66|
|[2.3600345733874573,3.1542431036925693,2.083476457489454,2.0547931158599058,0.0,2.1549470973741336,0.0,2.225805748475487]|3.5 |
|[2.659953293565682,2.1028287357950464,2.083476457489454,2.0547931158599058,0.0,2.1549470973741336,0.0,2.225805748475487] |3.31|
|[2.762172782465377,4.205657471590093,0.0,2.0547931158599058,0.0,2.1549470973741336,0.0,2.2258057

In [28]:
# 전체 데이터의 75%는 학습 데이터로, 25%는 테스트 데이터로 사용
train_data, test_data = data.randomSplit([0.75, 0.25])

# 분할된 학습 데이터와 테스트 데이터의 개수를 출력
train_data.count(), test_data.count()

(188, 56)

In [29]:
# 선형 회귀(Linear Regression) 모델을 생성하고 학습
# featuresCol: 독립 변수(특징 벡터)가 저장된 컬럼
# labelCol: 종속 변수(타겟 값)가 저장된 컬럼
regressor = LinearRegression(featuresCol="standardized", labelCol='tip')

# 선형 회귀 모델을 학습(fit). 학습 결과는 model 변수에 저장
model = regressor.fit(train_data)

In [30]:
# 모델의 계수와 절편 출력
print(f"Coefficients: {model.coefficients}")
print(f"Intercept: {model.intercept}")

# 모델 요약 정보 확인
summary = model.summary
print(f"R-squared: {summary.r2}")
print(f"Mean Squared Error: {summary.meanSquaredError}")
print(f"Root Mean Squared Error: {summary.rootMeanSquaredError}")

Coefficients: [0.8687387826918134,0.17781445137889068,-0.016987525524174988,0.01854760111561413,-0.01973038500753532,-0.07006087827670157,-0.026598149365583162,-0.041285448282551034]
Intercept: 0.7033006655721113
R-squared: 0.5137228806966799
Mean Squared Error: 0.9349797263968816
Root Mean Squared Error: 0.9669434970032539


In [31]:
# 학습된 선형 회귀 모델을 사용하여 테스트 데이터에 대한 예측을 수행
# transform 메서드는 테스트 데이터셋(test_data)을 입력으로 받아 각 샘플의 예측값을 계산
# 결과는 pred 변수에 저장되며, 원본 데이터와 예측값이 포함된 새로운 데이터프레임 생성
pred = model.transform(test_data)
pred.limit(5)

standardized,tip,prediction
"(8,[0,1,2,7],[3.0...",4.0,3.6120368435152406
"(8,[0,1,3,6],[1.1...",1.5,2.0936317987433664
"(8,[0,1,3,7],[2.5...",3.25,3.2434829573961665
"(8,[0,1,4,7],[1.6...",4.0,2.34064920917669
"(8,[0,1,4,7],[2.5...",3.48,3.132060746112681
